# Karger's Algorithm on the 5

In [ ]:
import pandas as pd
import networkx as nx
import random
import copy
import time
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
def load_graph_from_csv(file_path):
    df = pd.read_csv(file_path)
    G = nx.Graph()
    for _, row in df.iterrows():
        u, v, w = row['source'], row['target'], float(row['weight'])
        G.add_edge(u, v, weight=w)
    return G

In [ ]:
def load_graph_from_csv_no_header(file_path):
    df = pd.read_csv(file_path, header=None)
    G = nx.Graph()
    for _, row in df.iterrows():
        u = row[0]
        v = row[1]
        try:
            w = float(row[2])
            if w == 0:
                w = 1.0
        except:
            w = 1.0
        G.add_edge(u, v, weight=w)
    return G

In [ ]:
def karger_min_cut(G):
    graph = copy.deepcopy(G)

    while len(graph.nodes) > 2:
        u, v = random.choice(list(graph.edges()))
        for neighbor in list(graph.neighbors(v)):
            if neighbor != u:
                # Access weight from the contracted graph
                weight = graph[v][neighbor]['weight']
                graph.add_edge(u, neighbor, weight=weight)
        graph.remove_node(v)
        graph.remove_edges_from(nx.selfloop_edges(graph))

    return graph.size(weight='weight')

In [ ]:
def karger_with_visualization(G, capture_every=10):
    graph = copy.deepcopy(G)
    steps = []
    step_num = 0

    while len(graph.nodes) > 2:
        u, v = random.choice(list(graph.edges()))

        if step_num % capture_every == 0:
            steps.append((copy.deepcopy(graph), [(u, v)]))

        for neighbor in list(graph.neighbors(v)):
            if neighbor != u:
                # Access weight from the contracted graph
                weight = graph[v][neighbor]['weight']
                graph.add_edge(u, neighbor, weight=weight)

        graph.remove_node(v)
        graph.remove_edges_from(nx.selfloop_edges(graph))
        step_num += 1

    steps.append((graph, []))
    return graph, steps

In [ ]:
def run_karger_repeatedly(graph, trials):
    min_cut = float('inf')
    best_trial = -1
    start_time = time.time()

    for i in range(trials):
        cut_value, _ = karger_min_cut(graph)  # Get only the cut value
        if cut_value < min_cut:
            min_cut = cut_value
            best_trial = i

    end_time = time.time()
    return {
        'min_cut': min_cut,
        'best_trial': best_trial,
        'execution_time': end_time - start_time
    }

visualizing the graph on a sample (because dataset is too large)

In [ ]:
def get_sampled_connected_subgraph(G, sample_size):
    # Pick the first N nodes
    sample_nodes = list(G.nodes)[:sample_size]
    H = G.subgraph(sample_nodes).copy()

    # Ensure the sample is connected
    if not nx.is_connected(H):
        H = H.subgraph(max(nx.connected_components(H), key=len)).copy()

    return H

In [ ]:
def visualize_graph(G, title="Graph", contracted_edges=[]):
    pos = nx.spring_layout(G, seed=42)

    plt.figure(figsize=(10, 7))
    nx.draw(G, pos, node_color='lightblue', with_labels=False, node_size=30, edge_color='gray')

    if contracted_edges:
        nx.draw_networkx_edges(G, pos, edgelist=contracted_edges, edge_color='red', width=2)

    plt.title(title)
    plt.show()


In [ ]:
def show_contraction_steps(steps):
    for i, (G_step, contracted) in enumerate(steps):
        visualize_graph(G_step, title=f"Step {i+1}", contracted_edges=contracted)

In [ ]:
def animate_contractions(steps):
    fig, ax = plt.subplots(figsize=(8, 6))

    def update(i):
        ax.clear()
        G_step, contracted = steps[i]
        pos = nx.spring_layout(G_step, seed=42)
        nx.draw(G_step, pos, node_color='skyblue', node_size=60, edge_color='gray', ax=ax, with_labels=False)
        if contracted:
            nx.draw_networkx_edges(G_step, pos, edgelist=contracted, edge_color='red', width=2)
        ax.set_title(f"Step {i+1}")
        ax.axis('off')

    ani = animation.FuncAnimation(fig, update, frames=len(steps), interval=1000, repeat=False)
    return ani

## soc-sign-bitcoinalpha


*   3,783 nodes
*   24,186 edges



In [ ]:
file_path = "soc-sign-bitcoinalpha_cleaned.csv"
G = load_graph_from_csv(file_path)

# Ensure connected
if not nx.is_connected(G):
    G = G.subgraph(max(nx.connected_components(G), key=len)).copy()

# Run with 1 trial
start_time = time.time()
min_cut_value = karger_min_cut(G)
end_time = time.time()
runtime1 = end_time - start_time

# Output
print(f" Number of Nodes in Connected Graph: {G.number_of_nodes()}")
print(f" Number of Edges  in Connected Graph: {G.number_of_edges()}")
print(f"Min-Cut Value: {min_cut_value}")
print(f"Runtime: {runtime1:.2f} seconds")

visualizing the graph

In [ ]:
visualize_graph(G, title="Visualization of BitcoinAlpha (connected only)")

taking a sample and creating visualization (video)

In [ ]:
# Create the sampled graph
sampled_G = get_sampled_connected_subgraph(G, sample_size=500)

# Run Karger and capture visual steps
final_sample_graph, visual_steps = karger_with_visualization(sampled_G, capture_every=10)

# Show the contraction steps
show_contraction_steps(visual_steps)

In [ ]:
ani = animate_contractions(visual_steps)
#commented this to minimize file size
#ani.save("karger_contraction_video.mp4", writer='ffmpeg')

## soc-sign-bitcoinotc

* 5,881 nodes
* 35,592 edges

In [ ]:
from typing_extensions import runtime
file_path = "soc-sign-bitcoinotc_cleaned.csv"
G2 = load_graph_from_csv(file_path)

# Ensure connected
if not nx.is_connected(G2):
    G2 = G2.subgraph(max(nx.connected_components(G2), key=len)).copy()

# Run with 1 trial
start_time = time.time()
min_cut_value = karger_min_cut(G2)
end_time = time.time()
runtime2 = end_time - start_time

# Output
print(f" Number of Nodes in Connected Graph: {G2.number_of_nodes()}")
print(f" Number of Edges  in Connected Graph: {G2.number_of_edges()}")
print(f"Min-Cut Value: {min_cut_value}")
print(f"Runtime: {runtime2:.2f} seconds")

visualizing the graph

In [ ]:
visualize_graph(G2, title="Visualization of BitcoinOTC (connected only)")

taking a sample and creating a visualization (video)

In [ ]:
# Create the sampled graph
sampled_G2 = get_sampled_connected_subgraph(G2, sample_size=500)

# Run Karger and capture visual steps
final_sample_graph2, visual_steps2 = karger_with_visualization(sampled_G2, capture_every=10)

# Show the contraction steps
show_contraction_steps(visual_steps2)

In [ ]:
ani2 = animate_contractions(visual_steps2)
#ani2.save("karger_contraction_bitcoinotc_video.mp4", writer='ffmpeg')

## soc-advogato


*   6,541 nodes
*   51,127 edges



In [ ]:
file_path = 'soc-advogato_cleaned.csv'
G3 = load_graph_from_csv_no_header(file_path)

# Ensure connected
if not nx.is_connected(G3):
    G3 = G3.subgraph(max(nx.connected_components(G3), key=len)).copy()

# Run with 1 trial
start_time = time.time()
min_cut_value = karger_min_cut(G3)
end_time = time.time()
runtime3 = end_time - start_time

# Output
print(f" Number of Nodes in Connected Graph: {G3.number_of_nodes()}")
print(f" Number of Edges  in Connected Graph: {G3.number_of_edges()}")
print(f"Min-Cut Value: {min_cut_value}")
print(f"Runtime: {runtime3:.2f} seconds")

visualizing the graoh

In [ ]:
visualize_graph(G3, title="Visualization of Advogato (connected only)")

taking a sample and creating a visualization (video)

In [ ]:
# Create the sampled graph
sampled_G3 = get_sampled_connected_subgraph(G3, sample_size=500)

# Run Karger and capture visual steps
final_sample_graph3, visual_steps3 = karger_with_visualization(sampled_G3, capture_every=10)

# Show the contraction steps
show_contraction_steps(visual_steps3)

In [ ]:
ani3 = animate_contractions(visual_steps3)
#ani3.save("karger_contraction_advogato_video.mp4", writer='ffmpeg')

## soc-epinions
* 75,879 nodes
* 508,837 edges

In [ ]:
file_path = 'soc-epinions_cleaned.csv'
G4 = load_graph_from_csv(file_path)

# Ensure connected
if not nx.is_connected(G4):
    G4 = G4.subgraph(max(nx.connected_components(G4), key=len)).copy()

# Run with 1 trial
start_time = time.time()
min_cut_value = karger_min_cut(G4)
end_time = time.time()
runtime4 = end_time - start_time

# Output
print(f" Number of Nodes in Connected Graph: {G4.number_of_nodes()}")
print(f" Number of Edges  in Connected Graph: {G4.number_of_edges()}")
print(f"Min-Cut Value: {min_cut_value}")
print(f"Runtime: {runtime4:.2f} seconds")

taking a sample and creating a visualization (video)

In [ ]:
# Create the sampled graph
sampled_G4 = get_sampled_connected_subgraph(G4, sample_size=500)

# Run Karger and capture visual steps
final_sample_graph4, visual_steps4 = karger_with_visualization(sampled_G4, capture_every=10)

# Show the contraction steps
show_contraction_steps(visual_steps4)

In [ ]:
ani4 = animate_contractions(visual_steps4)
#ani4.save("karger_contraction_epinions_video.mp4", writer='ffmpeg')

## soc-LiveMocha
* 104,103 nodes
* 2,193,083 edges

- tried executing the algorithm on the full dataset but it took 6 hours only to crash at the end
- second attempt to take only 10k nodes

In [ ]:
file_path = 'soc-livemocha_cleaned.csv'
G5 = load_graph_from_csv(file_path)

# Sort nodes by degree and take top 10k
degrees = sorted(G.degree, key=lambda x: x[1], reverse=True)
top_nodes = [node for node, deg in degrees[:10000]]
G10k = G.subgraph(top_nodes).copy()

# Ensure it's connected
if not nx.is_connected(G10k):
    G10k = G10k.subgraph(max(nx.connected_components(G10k), key=len)).copy()

print(f"Using top 10k node subgraph: {G10k.number_of_nodes()} nodes, {G10k.number_of_edges()} edges")

start_time = time.time()
min_cut_value = karger_min_cut(G10k)
end_time = time.time()
runtime5 = end_time - start_time

# Output

print(f"Min-Cut (LiveMocha 10k nodes): {min_cut_value}")
print(f"Runtime: {runtime5:.2f} seconds")

visualization on sample 500

In [ ]:
# Create the sampled graph
sampled_G5 = get_sampled_connected_subgraph(G5, sample_size=500)

# Run Karger and capture visual steps
final_sample_graph5, visual_steps5 = karger_with_visualization(sampled_G5, capture_every=10)

# Show the contraction steps
show_contraction_steps(visual_steps5)

In [ ]:
ani5 = animate_contractions(visual_steps5)
HTML(ani5.to_jshtml())  # Inline preview in Jupyter Notebook
ani5.save("karger_contraction_livemocha_video.mp4", writer='ffmpeg')

## 3d graph to visualize how number of nodes and edges affects the runtime

In [ ]:
# Collect data
runtimes = [runtime1, runtime2, runtime3, runtime4, runtime5] # Assuming runtime5 is for the 10k node LiveMocha graph
nodes = [G.number_of_nodes(), G2.number_of_nodes(), G3.number_of_nodes(), G4.number_of_nodes(), G10k.number_of_nodes()]
edges = [G.number_of_edges(), G2.number_of_edges(), G3.number_of_edges(), G4.number_of_edges(), G10k.number_of_edges()]
dataset_names = ['BitcoinAlpha', 'BitcoinOTC', 'Advogato', 'Epinions', 'LiveMocha (10k Nodes)']

# Create the 3D plot
fig = plt.figure(figsize=(16, 12))
ax = fig.add_subplot(111, projection='3d')

# Plot the points
ax.scatter(nodes, edges, runtimes, c=runtimes, cmap='viridis', marker='o', s=100)

# Label the points
for i in range(len(dataset_names)):
    ax.text(nodes[i], edges[i], runtimes[i], dataset_names[i], zdir='z')

# Connect the dots with lines (based on the order of the datasets)
ax.plot(nodes, edges, runtimes, color='gray', linestyle='-', alpha=0.5)

# Set labels and title
ax.set_xlabel('Number of Nodes')
ax.set_ylabel('Number of Edges')
ax.set_zlabel('Runtime (seconds)')
ax.set_title('Karger\'s Algorithm Runtime vs. Graph Size (Nodes and Edges)')

plt.show()


In [ ]:
datasets = ['BitcoinAlpha', 'BitcoinOTC', 'Advogato', 'Epinions', 'LiveMocha (10k Nodes)']
runtimes_bar = [runtime1, runtime2, runtime3, runtime4, runtime5]

plt.figure(figsize=(12, 8))
plt.bar(datasets, runtimes_bar, color=['skyblue', 'lightgreen', 'salmon', 'gold', 'purple'])
plt.ylabel('Runtime (seconds)')
plt.title('Karger\'s Algorithm Runtime for Different Datasets (1 Trial)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()
